<a href="https://colab.research.google.com/github/parthpendal/Pyspark/blob/main/Pyspark_6_(Mlib).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pyspark

In [21]:
from pyspark.sql import SparkSession

In [22]:
spark=SparkSession.builder.appName("session1").getOrCreate()

In [23]:
df_spark = spark.read.csv("/content/Emp_exp_sal.csv",header=True,inferSchema=True)

In [24]:
df_spark.show()

+-----+---+----------+------+
| Name|Age|Expereince|Salary|
+-----+---+----------+------+
| John| 20|         2|  4000|
|Jacob| 24|         3| 45000|
| Mike| 30|         4| 50000|
| Asha| 50|        19|110000|
|Taryn| 35|         7| 60000|
|David| 62|        30|250000|
|  Tim| 25|         8| 65000|
+-----+---+----------+------+



In [25]:
df_spark.orderBy("Expereince").show()

+-----+---+----------+------+
| Name|Age|Expereince|Salary|
+-----+---+----------+------+
| John| 20|         2|  4000|
|Jacob| 24|         3| 45000|
| Mike| 30|         4| 50000|
|Taryn| 35|         7| 60000|
|  Tim| 25|         8| 65000|
| Asha| 50|        19|110000|
|David| 62|        30|250000|
+-----+---+----------+------+



In [27]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['Age','Expereince'],outputCol="Independent Features")

In [29]:
output=featureassembler.transform(df_spark)

In [30]:
output.show()

+-----+---+----------+------+--------------------+
| Name|Age|Expereince|Salary|Independent Features|
+-----+---+----------+------+--------------------+
| John| 20|         2|  4000|          [20.0,2.0]|
|Jacob| 24|         3| 45000|          [24.0,3.0]|
| Mike| 30|         4| 50000|          [30.0,4.0]|
| Asha| 50|        19|110000|         [50.0,19.0]|
|Taryn| 35|         7| 60000|          [35.0,7.0]|
|David| 62|        30|250000|         [62.0,30.0]|
|  Tim| 25|         8| 65000|          [25.0,8.0]|
+-----+---+----------+------+--------------------+



In [31]:
finalized_data=output.select(['Salary','Independent Features'])

In [33]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data,test_data=(finalized_data.randomSplit([0.75,0.25]))
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)

In [34]:
regressor.coefficients

DenseVector([522.2742, 7835.248])

In [35]:
regressor.intercept

-15854.898438362778

In [36]:
predicted_results=regressor.evaluate(test_data)

In [38]:
predicted_results.predictions.show()

+------+--------------------+------------------+
|Salary|Independent Features|        prediction|
+------+--------------------+------------------+
| 45000|          [24.0,3.0]|20185.426811011872|
| 50000|          [30.0,4.0]|31154.320112198708|
|110000|         [50.0,19.0]|159128.52392883616|
+------+--------------------+------------------+



In [39]:
predicted_results.meanAbsoluteError,predicted_results.meanSquaredError

(30929.592335208526, 1128178185.4704514)